<a href="https://colab.research.google.com/github/piyush-an/INFO7374_Predict_StockPrice/blob/main/notebooks/2_FeatureMart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part 2: Using the features/factors you take and discovered, e.g., FRED, Fama-French website, ADS, AR, CAPM, momentum factors, volume, price/return lags, etc.) to construct a feature database
- The target variable Y can be either price or return
- Frequency could be either daily or monthly

## Install Dependency and files

In [1]:
%%bash

pip install pandas yfinance ta seaborn matplotlib pandas-datareader jinja2 fredapi openpyxl xgboost scikit-learn statsmodels mlflow

if [ ! -f "ads_index_most_current_vintage.xlsx" ]; then
    wget https://www.philadelphiafed.org/-/media/frbp/assets/surveys-and-data/ads/ads_index_most_current_vintage.xlsx
fi
if [ ! -f "F-F_Research_Data_Factors_daily.CSV" ]; then
  wget https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_Factors_daily_CSV.zip
	unzip F-F_Research_Data_Factors_daily_CSV.zip
fi

In [3]:
import numpy as np
import pandas as pd
from datetime import datetime
import yfinance as yf
import ta

In [4]:
# Define dataset start and end date => Two years worth of data
start_date = datetime(2018, 1, 1)
end_date = datetime(2023, 12, 31)

# Downloaded data
NVDA_STOCK = yf.download("NVDA", start_date, end_date)
NVDA_STOCK.describe()

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
count,1497.000000,1497.000000,1497.000000,1497.000000,1497.000000,1.497000e+03
mean,154.860705,157.730720,151.894709,154.939529,154.659071,4.778147e+07
std,119.083942,121.095086,116.846874,119.041880,119.121029,2.153778e+07
min,31.622499,32.494999,31.115000,31.770000,31.523230,9.788400e+06
25%,59.767502,60.732498,58.952499,59.882500,59.564747,3.377800e+07
50%,130.880005,133.372498,129.067505,131.279999,131.065811,4.400920e+07
75%,209.399994,212.059998,206.500000,208.160004,207.811203,5.804240e+07
max,502.160004,505.480011,492.220001,504.089996,504.045685,2.511528e+08


In [5]:
NVDA_STOCK.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-01-02,48.945000,49.875000,48.625000,49.837502,49.312790,35561600
2018-01-03,51.025002,53.424999,50.937500,53.117500,52.558270,91470400
2018-01-04,53.939999,54.512501,53.172501,53.397499,52.835316,58326800
2018-01-05,53.547501,54.227501,52.770000,53.849998,53.283039,58012400
2018-01-08,55.099998,56.250000,54.645000,55.500000,54.915672,88121600


In [6]:
NVDA_STOCK["Returns"] = NVDA_STOCK["Adj Close"] - NVDA_STOCK["Adj Close"].shift(1)
NVDA_STOCK.head()

,Open,High,Low,Close,Adj Close,Volume,Returns
Date,,,,,,,
2018-01-02,48.945000,49.875000,48.625000,49.837502,49.312790,35561600,NaN
2018-01-03,51.025002,53.424999,50.937500,53.117500,52.558270,91470400,3.245480
2018-01-04,53.939999,54.512501,53.172501,53.397499,52.835316,58326800,0.277046
2018-01-05,53.547501,54.227501,52.770000,53.849998,53.283039,58012400,0.447723
2018-01-08,55.099998,56.250000,54.645000,55.500000,54.915672,88121600,1.632633


Calculating daily returns:

Produce the day's difference of the stock dataframe: (`np.log(nvda['Open']) - np.log(nvda['Open'].shift(+1))`)

When we take the logarithm of the ratio between today's closing price and yesterday's, we're essentially computing the daily percentage change in the stock price. Using logarithms in return calculations helps us handle the additive nature of log-returns, making overall return calculations more interpretable and facilitating mathematical operations.

In [7]:
# Daily return
NVDA_STOCK["Daily_Return"] = np.log(NVDA_STOCK["Adj Close"]) - np.log(NVDA_STOCK["Adj Close"].shift(1))
NVDA_STOCK = NVDA_STOCK.dropna()
NVDA_STOCK.head()

,Open,High,Low,Close,Adj Close,Volume,Returns,Daily_Return
Date,,,,,,,,
2018-01-03,51.025002,53.424999,50.937500,53.117500,52.558270,91470400,3.245480,0.063739
2018-01-04,53.939999,54.512501,53.172501,53.397499,52.835316,58326800,0.277046,0.005257
2018-01-05,53.547501,54.227501,52.770000,53.849998,53.283039,58012400,0.447723,0.008438
2018-01-08,55.099998,56.250000,54.645000,55.500000,54.915672,88121600,1.632633,0.030181
2018-01-09,55.555000,55.955002,54.660000,55.485001,54.900837,49700000,-0.014835,-0.000270


### Feature-set 1: Typical Price, Typical_Price_Return
- `Typical_Price` is the *mean* value of High, Low and Close values

In [8]:
NVDA_STOCK["Typical_Price"] = NVDA_STOCK[["High", "Low", "Close"]].mean(axis=1)
NVDA_STOCK["Typical_Price_Return"] = (
    np.log(NVDA_STOCK.Typical_Price) - np.log(NVDA_STOCK.Typical_Price.shift(+1))
) * 100.0
NVDA_STOCK = NVDA_STOCK.dropna()
NVDA_STOCK.head()

/tmp/ipykernel_93225/2203751036.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  NVDA_STOCK["Typical_Price"] = NVDA_STOCK[["High", "Low", "Close"]].mean(axis=1)
/tmp/ipykernel_93225/2203751036.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  NVDA_STOCK["Typical_Price_Return"] = (


,Open,High,Low,Close,Adj Close,Volume,Returns,Daily_Return,Typical_Price,Typical_Price_Return
Date,,,,,,,,,,
2018-01-04,53.939999,54.512501,53.172501,53.397499,52.835316,58326800,0.277046,0.005257,53.694167,2.261820
2018-01-05,53.547501,54.227501,52.770000,53.849998,53.283039,58012400,0.447723,0.008438,53.615833,-0.145995
2018-01-08,55.099998,56.250000,54.645000,55.500000,54.915672,88121600,1.632633,0.030181,55.465000,3.390777
2018-01-09,55.555000,55.955002,54.660000,55.485001,54.900837,49700000,-0.014835,-0.000270,55.366667,-0.177445
2018-01-10,54.549999,55.955002,54.000000,55.919998,55.331257,58266400,0.430420,0.007809,55.291667,-0.135554


### Feature-set 2: Common Transforms
- `log of volume`
- `pct_change of volume`
- `difference in volume`
- `log of 5 day moving average of volume`
- `Daily volume vs. 200 day moving average`
- `Daily closing price vs. 50 day exponential moving average`

In [9]:
NVDA_STOCK["Volume_Log"] = np.log(NVDA_STOCK.Volume)
NVDA_STOCK["Volume_Differencing"] = NVDA_STOCK.Volume.diff()
NVDA_STOCK["Volume_Differencing_10"] = NVDA_STOCK.Volume.diff(10)
NVDA_STOCK["Volumne_Percent_Change"] = NVDA_STOCK.Volume.pct_change()

In [10]:
# Log of 5 day moving average of volume
NVDA_STOCK["MA_5"] = np.log(NVDA_STOCK.Volume.rolling(5).mean())

# Daily volume vs. 200 day moving average
NVDA_STOCK["Volumne_MA_200"] = (
    NVDA_STOCK.Volume / NVDA_STOCK.Volume.rolling(200).mean() - 1
)

# Daily closing price vs. 50 day Exponential Moving Avg
NVDA_STOCK["Close_EMA_50"] = NVDA_STOCK.Close / NVDA_STOCK.Close.ewm(span=50).mean() - 1

In [11]:
NVDA_STOCK.head()

,Open,High,Low,Close,Adj Close,Volume,Returns,Daily_Return,Typical_Price,Typical_Price_Return,Volume_Log,Volume_Differencing,Volume_Differencing_10,Volumne_Percent_Change,MA_5,Volumne_MA_200,Close_EMA_50
Date,,,,,,,,,,,,,,,,,
2018-01-04,53.939999,54.512501,53.172501,53.397499,52.835316,58326800,0.277046,0.005257,53.694167,2.261820,17.881572,NaN,NaN,NaN,NaN,NaN,0.000000
2018-01-05,53.547501,54.227501,52.770000,53.849998,53.283039,58012400,0.447723,0.008438,53.615833,-0.145995,17.876167,-314400.0,NaN,-0.005390,NaN,NaN,0.004134
2018-01-08,55.099998,56.250000,54.645000,55.500000,54.915672,88121600,1.632633,0.030181,55.465000,3.390777,18.294228,30109200.0,NaN,0.519013,NaN,NaN,0.022527
2018-01-09,55.555000,55.955002,54.660000,55.485001,54.900837,49700000,-0.014835,-0.000270,55.366667,-0.177445,17.721515,-38421600.0,NaN,-0.436007,NaN,NaN,0.016254
2018-01-10,54.549999,55.955002,54.000000,55.919998,55.331257,58266400,0.430420,0.007809,55.291667,-0.135554,17.880536,8566400.0,NaN,0.172362,17.950444,NaN,0.018883


### Feature-set 3: Momentum Indicators

**1. AwesomeOscillatorIndicator**

In [12]:
NVDA_STOCK['Momentum_AwesomeOscillatorIndicator'] = ta.momentum.AwesomeOscillatorIndicator(NVDA_STOCK.High, NVDA_STOCK.Low,window1 = 5,window2 = 34, fillna=False).awesome_oscillator()

**2. Kaufman’s Adaptive Moving Average (KAMA)**

In [13]:
NVDA_STOCK['Momentum_KAMA'] = ta.momentum.KAMAIndicator(NVDA_STOCK.Close, fillna=False).kama()

**3. PercentagePriceOscillator**

In [14]:
NVDA_STOCK['Momentum_PercentagePVolumneOscillator'] = ta.momentum.PercentageVolumeOscillator(NVDA_STOCK.Volume, fillna=False).pvo()

**4. Rate of Change (ROC)**

In [15]:
NVDA_STOCK['Momentum_ROC'] = ta.momentum.ROCIndicator(NVDA_STOCK.Close, fillna=False).roc()

**5. Relative Strength Index (RSI)**

In [16]:
NVDA_STOCK['Momentum_RSI'] = ta.momentum.RSIIndicator(NVDA_STOCK.Close, fillna=False).rsi()

**6. Stochastic RSI**

In [17]:
NVDA_STOCK['Momentum_StochRSIIndicator'] = ta.momentum.StochRSIIndicator(NVDA_STOCK.Close, fillna=False).stochrsi()

**7. True strength index (TSI)**

In [18]:
NVDA_STOCK['Momentum_TSIIndicator'] = ta.momentum.TSIIndicator(NVDA_STOCK.Close, fillna=False).tsi()

In [19]:
NVDA_STOCK.head()

,Open,High,Low,Close,Adj Close,Volume,Returns,Daily_Return,Typical_Price,Typical_Price_Return,...,MA_5,Volumne_MA_200,Close_EMA_50,Momentum_AwesomeOscillatorIndicator,Momentum_KAMA,Momentum_PercentagePVolumneOscillator,Momentum_ROC,Momentum_RSI,Momentum_StochRSIIndicator,Momentum_TSIIndicator
Date,,,,,,,,,,,,,,,,,,,,,
2018-01-04,53.939999,54.512501,53.172501,53.397499,52.835316,58326800,0.277046,0.005257,53.694167,2.261820,...,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-05,53.547501,54.227501,52.770000,53.849998,53.283039,58012400,0.447723,0.008438,53.615833,-0.145995,...,NaN,NaN,0.004134,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-08,55.099998,56.250000,54.645000,55.500000,54.915672,88121600,1.632633,0.030181,55.465000,3.390777,...,NaN,NaN,0.022527,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-09,55.555000,55.955002,54.660000,55.485001,54.900837,49700000,-0.014835,-0.000270,55.366667,-0.177445,...,NaN,NaN,0.016254,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-10,54.549999,55.955002,54.000000,55.919998,55.331257,58266400,0.430420,0.007809,55.291667,-0.135554,...,17.950444,NaN,0.018883,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Feature-set 4: Trend Indicators

**1. Average Directional Movement Index (ADX)**

In [20]:
NVDA_STOCK['Trend_ADX'] = ta.trend.ADXIndicator(NVDA_STOCK.High,NVDA_STOCK.Low, NVDA_STOCK.Close, window = 20,fillna=False).adx()

**2. Aroon Indicator**

In [21]:
NVDA_STOCK['Trend_AroonIndicator'] = ta.trend.AroonIndicator(NVDA_STOCK.Close, NVDA_STOCK.Low, window=20, fillna=False).aroon_indicator()

**3. Commodity Channel Index (CCI)**

In [22]:
NVDA_STOCK['Trend_CCI'] = ta.trend.CCIIndicator(NVDA_STOCK.High, NVDA_STOCK.Low, NVDA_STOCK.Close, window = 20,fillna=False).cci()

**4. Detrended Price Oscillator (DPO)**

In [23]:
NVDA_STOCK['Trend_DPO'] = ta.trend.DPOIndicator(NVDA_STOCK.Close, window = 20, fillna=False).dpo()

**5. EMA - Exponential Moving Average**

In [24]:
NVDA_STOCK['Trend_EMA'] = ta.trend.EMAIndicator(NVDA_STOCK.Close, window = 20, fillna=False).ema_indicator()

**6. Moving Average Convergence Divergence (MACD)**

In [25]:
NVDA_STOCK['Trend_MACD'] = ta.trend.MACD(NVDA_STOCK.Close, fillna=False).macd()

**7. Mass Index (MI)**

In [26]:
NVDA_STOCK['Trend_MI'] = ta.trend.MassIndex(NVDA_STOCK.High, NVDA_STOCK.Low, fillna=False).mass_index()

In [27]:
NVDA_STOCK.head()

,Open,High,Low,Close,Adj Close,Volume,Returns,Daily_Return,Typical_Price,Typical_Price_Return,...,Momentum_RSI,Momentum_StochRSIIndicator,Momentum_TSIIndicator,Trend_ADX,Trend_AroonIndicator,Trend_CCI,Trend_DPO,Trend_EMA,Trend_MACD,Trend_MI
Date,,,,,,,,,,,,,,,,,,,,,
2018-01-04,53.939999,54.512501,53.172501,53.397499,52.835316,58326800,0.277046,0.005257,53.694167,2.261820,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-05,53.547501,54.227501,52.770000,53.849998,53.283039,58012400,0.447723,0.008438,53.615833,-0.145995,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-08,55.099998,56.250000,54.645000,55.500000,54.915672,88121600,1.632633,0.030181,55.465000,3.390777,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-09,55.555000,55.955002,54.660000,55.485001,54.900837,49700000,-0.014835,-0.000270,55.366667,-0.177445,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-10,54.549999,55.955002,54.000000,55.919998,55.331257,58266400,0.430420,0.007809,55.291667,-0.135554,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN


### Feature-set 5: Volumne Indicator

**1. Chaikin Money Flow (CMF)**

In [28]:
NVDA_STOCK['Volumne_CMF'] = ta.volume.ChaikinMoneyFlowIndicator(NVDA_STOCK.High,NVDA_STOCK.Low,NVDA_STOCK.Close, NVDA_STOCK.Volume,window = 20,fillna=False).chaikin_money_flow()

**2. Ease of movement (EoM, EMV)**

In [29]:
NVDA_STOCK['Volumne_EOM'] = ta.volume.EaseOfMovementIndicator(NVDA_STOCK.High,NVDA_STOCK.Low, NVDA_STOCK.Volume,window = 20,fillna=False).ease_of_movement()

**3. Force Index (FI)**

In [30]:
NVDA_STOCK['Volumne_FI'] = ta.volume.ForceIndexIndicator(NVDA_STOCK.Close, NVDA_STOCK.Volume,window = 20,fillna=False).force_index()

**4. Money Flow Index (MFI)**

In [31]:
NVDA_STOCK['Volumne_MFI'] = ta.volume.money_flow_index(NVDA_STOCK.High, NVDA_STOCK.Low, NVDA_STOCK.Close,NVDA_STOCK.Volume, window=20, fillna=False)

**5. Volume Weighted Average Price (VWAP)**

In [32]:
NVDA_STOCK['Volumne_VWAP'] = ta.volume.VolumeWeightedAveragePrice(NVDA_STOCK.High, NVDA_STOCK.Low, NVDA_STOCK.Close,NVDA_STOCK.Volume, window=20, fillna=False).volume_weighted_average_price()

In [33]:
NVDA_STOCK.head()

,Open,High,Low,Close,Adj Close,Volume,Returns,Daily_Return,Typical_Price,Typical_Price_Return,...,Trend_CCI,Trend_DPO,Trend_EMA,Trend_MACD,Trend_MI,Volumne_CMF,Volumne_EOM,Volumne_FI,Volumne_MFI,Volumne_VWAP
Date,,,,,,,,,,,,,,,,,,,,,
2018-01-04,53.939999,54.512501,53.172501,53.397499,52.835316,58326800,0.277046,0.005257,53.694167,2.261820,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-05,53.547501,54.227501,52.770000,53.849998,53.283039,58012400,0.447723,0.008438,53.615833,-0.145995,...,NaN,NaN,NaN,NaN,NaN,NaN,-0.863636,NaN,NaN,NaN
2018-01-08,55.099998,56.250000,54.645000,55.500000,54.915672,88121600,1.632633,0.030181,55.465000,3.390777,...,NaN,NaN,NaN,NaN,NaN,NaN,3.549348,NaN,NaN,NaN
2018-01-09,55.555000,55.955002,54.660000,55.485001,54.900837,49700000,-0.014835,-0.000270,55.366667,-0.177445,...,NaN,NaN,NaN,NaN,NaN,NaN,-0.364788,NaN,NaN,NaN
2018-01-10,54.549999,55.955002,54.000000,55.919998,55.331257,58266400,0.430420,0.007809,55.291667,-0.135554,...,NaN,NaN,NaN,NaN,NaN,NaN,-1.107243,NaN,NaN,NaN


### Feature-set 6: Volatility Indicators

**1. Average True Range (ATR)**

In [34]:
NVDA_STOCK['Volatility_ATR'] = ta.volatility.AverageTrueRange(NVDA_STOCK.High, NVDA_STOCK.Low, NVDA_STOCK.Close, window=20, fillna=False).average_true_range()

**2. Bollinger Bands**

In [35]:
NVDA_STOCK['Volatility_BB'] = ta.volatility.BollingerBands(NVDA_STOCK.Close, window=20, fillna=False).bollinger_wband()

**3. Donchian Channel**

In [36]:
NVDA_STOCK['Volatility_DonchainChannel'] = ta.volatility.DonchianChannel(NVDA_STOCK.High, NVDA_STOCK.Low,NVDA_STOCK.Close, window=20, fillna=False).donchian_channel_wband()

**4. Ulcer Index**

In [37]:
NVDA_STOCK['Volatility_UlcerIndex'] = ta.volatility.UlcerIndex(NVDA_STOCK.Close, window=20, fillna=False).ulcer_index()

**5. Keltner channel (KC)**

In [38]:
NVDA_STOCK['Volatility_KeltnerChannel'] = ta.volatility.keltner_channel_hband(NVDA_STOCK.High, NVDA_STOCK.Low,NVDA_STOCK.Close, window=20, fillna=False)

In [39]:
NVDA_STOCK.dropna(inplace = True)

In [40]:
NVDA_STOCK.head()

,Open,High,Low,Close,Adj Close,Volume,Returns,Daily_Return,Typical_Price,Typical_Price_Return,...,Volumne_CMF,Volumne_EOM,Volumne_FI,Volumne_MFI,Volumne_VWAP,Volatility_ATR,Volatility_BB,Volatility_DonchainChannel,Volatility_UlcerIndex,Volatility_KeltnerChannel
Date,,,,,,,,,,,,,,,,,,,,,
2018-10-18,61.465000,61.852501,59.272499,59.882500,59.357967,52402000,-0.874779,-0.014630,60.335833,-1.370346,...,-0.166595,-3.975714,-2.515214e+07,45.203574,65.418863,2.260525,26.135972,22.298079,10.741495,67.986543
2018-10-19,60.439999,60.637501,56.924999,57.292500,56.790649,61360800,-2.567318,-0.044215,58.285000,-3.458141,...,-0.189750,-10.777065,-3.789236e+07,44.660537,64.892034,2.333124,28.543358,24.961590,11.595971,67.696292
2018-10-22,57.820000,58.830002,56.767502,57.805000,57.298664,36884400,0.508015,0.008906,57.800835,-0.834155,...,-0.219260,-5.493928,-3.248325e+07,44.509744,64.570526,2.319593,30.310946,25.371115,12.354732,67.307376
2018-10-23,55.107498,56.047501,54.177502,55.264999,54.780922,62643600,-2.517742,-0.044935,55.163334,-4.670472,...,-0.216777,-8.018833,-4.454340e+07,40.195354,63.871194,2.384988,32.945426,29.643461,13.380735,66.757917
2018-10-24,54.877499,55.347500,49.712502,49.852501,49.415833,88428800,-5.365089,-0.103071,51.637501,-6.605032,...,-0.269425,-16.456613,-8.588410e+07,35.541378,62.703614,2.547489,38.472874,37.093141,15.020365,66.224375


In [ ]:
NVDA_STOCK.info()

### Feature-set 7: Fama-French Indicators

In [41]:
df_fama = pd.read_csv("./F-F_Research_Data_Factors_daily.CSV", skiprows=3)
df_fama = df_fama.iloc[:-1]
df_fama.rename(columns={"Unnamed: 0": "Date"}, inplace=True)
df_fama["Date"] = pd.to_datetime(df_fama["Date"])
df_fama = df_fama[(df_fama["Date"] >= start_date) & (df_fama["Date"] <= end_date)]
fama = df_fama.set_index("Date")

In [42]:
fama.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1468 entries, 2018-01-02 to 2023-10-31
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Mkt-RF  1468 non-null   float64
 1   SMB     1468 non-null   float64
 2   HML     1468 non-null   float64
 3   RF      1468 non-null   float64
dtypes: float64(4)
memory usage: 57.3 KB


In [43]:
NVDA_STOCK = pd.concat([NVDA_STOCK, fama], axis=1)
NVDA_STOCK.dropna(inplace=True)
NVDA_STOCK

,Open,High,Low,Close,Adj Close,Volume,Returns,Daily_Return,Typical_Price,Typical_Price_Return,...,Volumne_VWAP,Volatility_ATR,Volatility_BB,Volatility_DonchainChannel,Volatility_UlcerIndex,Volatility_KeltnerChannel,Mkt-RF,SMB,HML,RF
Date,,,,,,,,,,,,,,,,,,,,,
2018-10-18,61.465000,61.852501,59.272499,59.882500,59.357967,52402000.0,-0.874779,-0.014630,60.335833,-1.370346,...,65.418863,2.260525,26.135972,22.298079,10.741495,67.986543,-1.54,-0.54,0.42,0.008
2018-10-19,60.439999,60.637501,56.924999,57.292500,56.790649,61360800.0,-2.567318,-0.044215,58.285000,-3.458141,...,64.892034,2.333124,28.543358,24.961590,11.595971,67.696292,-0.25,-1.33,0.71,0.008
2018-10-22,57.820000,58.830002,56.767502,57.805000,57.298664,36884400.0,0.508015,0.008906,57.800835,-0.834155,...,64.570526,2.319593,30.310946,25.371115,12.354732,67.307376,-0.38,0.48,-1.25,0.008
2018-10-23,55.107498,56.047501,54.177502,55.264999,54.780922,62643600.0,-2.517742,-0.044935,55.163334,-4.670472,...,63.871194,2.384988,32.945426,29.643461,13.380735,66.757917,-0.62,-0.10,-0.41,0.008
2018-10-24,54.877499,55.347500,49.712502,49.852501,49.415833,88428800.0,-5.365089,-0.103071,51.637501,-6.605032,...,62.703614,2.547489,38.472874,37.093141,15.020365,66.224375,-3.33,-0.93,0.77,0.008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-25,433.980011,436.500000,415.549988,417.790009,417.753265,39837900.0,-18.838348,-0.044107,423.279999,-2.386560,...,440.555158,16.208317,14.665447,15.080417,7.275335,455.716169,-1.58,-0.30,0.84,0.021
2023-10-26,418.529999,422.559998,398.799988,403.260010,403.224548,54100100.0,-14.528717,-0.035397,408.206665,-3.626032,...,439.196002,16.585902,16.474270,17.545335,7.403454,455.207336,-1.15,0.85,1.66,0.021
2023-10-27,411.299988,412.059998,400.149994,405.000000,404.964386,41678400.0,1.739838,0.004306,405.736664,-0.606924,...,437.801639,16.352107,17.957779,17.605263,7.669887,453.846170,-0.53,-0.35,-0.57,0.021


### Feature-set 8: Exracting external factors using Fred API

In [46]:
from fredapi import Fred

fred = Fred(api_key="dfd5e2b3a7c9a714430cdf450699e698")

In [47]:
# Japanese Yen to U.S. Dollar Spot Exchange Rate
# U.S. Dollars to Euro Spot Exchange Rate
# Coinbase Bitcoin
feat_list = ["SP500", "DEXJPUS", "DEXUSEU", "CBBTCUSD"]
feat_df = pd.DataFrame()
for feat in feat_list:
    feature = fred.get_series(feat, start_date, end_date)
    feature = feature.to_frame(feat)
    feature.dropna(inplace=True)
    feat_df = pd.concat([feat_df, feature], axis=1)
feat_df.dropna(inplace=True)
feat_df

,SP500,DEXJPUS,DEXUSEU,CBBTCUSD
2018-01-02,2695.81,112.18,1.2050,14781.51
2018-01-03,2713.06,112.28,1.2030,15098.14
2018-01-04,2723.99,112.78,1.2064,15144.99
2018-01-05,2743.15,113.18,1.2039,16960.01
2018-01-08,2747.71,113.08,1.1973,14993.74
...,...,...,...,...
2023-12-04,4569.78,147.15,1.0824,41856.31
2023-12-05,4567.18,147.26,1.0787,44060.06
2023-12-06,4549.34,147.16,1.0790,43778.51
2023-12-07,4585.59,144.10,1.0794,43305.91


In [48]:
NVDA_STOCK = pd.concat([NVDA_STOCK, feat_df], axis=1)
NVDA_STOCK.dropna(inplace=True)
NVDA_STOCK

,Open,High,Low,Close,Adj Close,Volume,Returns,Daily_Return,Typical_Price,Typical_Price_Return,...,Volatility_UlcerIndex,Volatility_KeltnerChannel,Mkt-RF,SMB,HML,RF,SP500,DEXJPUS,DEXUSEU,CBBTCUSD
2018-10-18,61.465000,61.852501,59.272499,59.882500,59.357967,52402000.0,-0.874779,-0.014630,60.335833,-1.370346,...,10.741495,67.986543,-1.54,-0.54,0.42,0.008,2768.78,112.11,1.1494,6394.96
2018-10-19,60.439999,60.637501,56.924999,57.292500,56.790649,61360800.0,-2.567318,-0.044215,58.285000,-3.458141,...,11.595971,67.696292,-0.25,-1.33,0.71,0.008,2767.78,112.52,1.1513,6382.99
2018-10-22,57.820000,58.830002,56.767502,57.805000,57.298664,36884400.0,0.508015,0.008906,57.800835,-0.834155,...,12.354732,67.307376,-0.38,0.48,-1.25,0.008,2755.88,112.78,1.1467,6407.65
2018-10-23,55.107498,56.047501,54.177502,55.264999,54.780922,62643600.0,-2.517742,-0.044935,55.163334,-4.670472,...,13.380735,66.757917,-0.62,-0.10,-0.41,0.008,2740.69,112.12,1.1480,6395.14
2018-10-24,54.877499,55.347500,49.712502,49.852501,49.415833,88428800.0,-5.365089,-0.103071,51.637501,-6.605032,...,15.020365,66.224375,-3.33,-0.93,0.77,0.008,2656.10,112.58,1.1389,6415.98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-25,433.980011,436.500000,415.549988,417.790009,417.753265,39837900.0,-18.838348,-0.044107,423.279999,-2.386560,...,7.275335,455.716169,-1.58,-0.30,0.84,0.021,4186.77,149.94,1.0583,34489.91
2023-10-26,418.529999,422.559998,398.799988,403.260010,403.224548,54100100.0,-14.528717,-0.035397,408.206665,-3.626032,...,7.403454,455.207336,-1.15,0.85,1.66,0.021,4137.23,150.44,1.0532,34140.00
2023-10-27,411.299988,412.059998,400.149994,405.000000,404.964386,41678400.0,1.739838,0.004306,405.736664,-0.606924,...,7.669887,453.846170,-0.53,-0.35,-0.57,0.021,4117.37,149.60,1.0592,33905.16
2023-10-30,410.869995,417.660004,404.809998,411.609985,411.573792,38802800.0,6.609406,0.016189,411.359996,1.376439,...,7.962946,452.096670,1.15,-0.39,0.28,0.021,4166.82,149.01,1.0620,34485.49


### Feature-set 8: ADS features

In [49]:
ads = pd.read_excel("ads_index_most_current_vintage.xlsx")
ads.rename(columns={"Unnamed: 0": "Date"}, inplace=True)
ads["Date"] = pd.to_datetime(ads["Date"], format="%Y:%m:%d")
ads = ads[(ads["Date"] >= start_date) & (ads["Date"] <= end_date)]
ads = ads.set_index("Date")
ads

,ADS_Index
Date,
2018-01-01,-0.260837
2018-01-02,-0.283936
2018-01-03,-0.304869
2018-01-04,-0.323642
2018-01-05,-0.340263
...,...
2023-11-28,-0.147276
2023-11-29,-0.143545
2023-11-30,-0.140216


In [50]:
ads.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2162 entries, 2018-01-01 to 2023-12-02
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ADS_Index  2162 non-null   float64
dtypes: float64(1)
memory usage: 33.8 KB


In [51]:
NVDA_STOCK = pd.concat([NVDA_STOCK, ads], axis=1)
NVDA_STOCK.dropna(inplace=True)
NVDA_STOCK

,Open,High,Low,Close,Adj Close,Volume,Returns,Daily_Return,Typical_Price,Typical_Price_Return,...,Volatility_KeltnerChannel,Mkt-RF,SMB,HML,RF,SP500,DEXJPUS,DEXUSEU,CBBTCUSD,ADS_Index
2018-10-18,61.465000,61.852501,59.272499,59.882500,59.357967,52402000.0,-0.874779,-0.014630,60.335833,-1.370346,...,67.986543,-1.54,-0.54,0.42,0.008,2768.78,112.11,1.1494,6394.96,-0.479664
2018-10-19,60.439999,60.637501,56.924999,57.292500,56.790649,61360800.0,-2.567318,-0.044215,58.285000,-3.458141,...,67.696292,-0.25,-1.33,0.71,0.008,2767.78,112.52,1.1513,6382.99,-0.478795
2018-10-22,57.820000,58.830002,56.767502,57.805000,57.298664,36884400.0,0.508015,0.008906,57.800835,-0.834155,...,67.307376,-0.38,0.48,-1.25,0.008,2755.88,112.78,1.1467,6407.65,-0.471768
2018-10-23,55.107498,56.047501,54.177502,55.264999,54.780922,62643600.0,-2.517742,-0.044935,55.163334,-4.670472,...,66.757917,-0.62,-0.10,-0.41,0.008,2740.69,112.12,1.1480,6395.14,-0.468534
2018-10-24,54.877499,55.347500,49.712502,49.852501,49.415833,88428800.0,-5.365089,-0.103071,51.637501,-6.605032,...,66.224375,-3.33,-0.93,0.77,0.008,2656.10,112.58,1.1389,6415.98,-0.464925
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-25,433.980011,436.500000,415.549988,417.790009,417.753265,39837900.0,-18.838348,-0.044107,423.279999,-2.386560,...,455.716169,-1.58,-0.30,0.84,0.021,4186.77,149.94,1.0583,34489.91,-0.529790
2023-10-26,418.529999,422.559998,398.799988,403.260010,403.224548,54100100.0,-14.528717,-0.035397,408.206665,-3.626032,...,455.207336,-1.15,0.85,1.66,0.021,4137.23,150.44,1.0532,34140.00,-0.521524
2023-10-27,411.299988,412.059998,400.149994,405.000000,404.964386,41678400.0,1.739838,0.004306,405.736664,-0.606924,...,453.846170,-0.53,-0.35,-0.57,0.021,4117.37,149.60,1.0592,33905.16,-0.511475
2023-10-30,410.869995,417.660004,404.809998,411.609985,411.573792,38802800.0,6.609406,0.016189,411.359996,1.376439,...,452.096670,1.15,-0.39,0.28,0.021,4166.82,149.01,1.0620,34485.49,-0.471021


Saving the final dataframe as the *feature mart.*

In [52]:
# timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
# filename = f'NVDA_feature_mart_{timestamp}.csv'
filename = f"NVDA_feature_mart.csv"
NVDA_STOCK.to_csv(filename, index=True)